In [ ]:
##Creates the Keyvault and certificate needed for the HPC deployment
# This is my own example!
# The more complete source is here - 
## https://raw.githubusercontent.com/Azure/hpcpack-template/master/Scripts/CreateHpcKeyVaultCertificate.ps1


In [22]:
# Account login IDs (not credentials) imported from a variables file

. ./AZ_Sub_Details_1.ps1

$tenantID = $TENANTID_DEF
$subscriptionID = $SUBSCRIPTIONID_DEF
$location="WestUS2"

$resourceGroupName="rg-prdw2hpc001"
$keyVaultName="kvt-prdw2hpc01"
$servPrincipalName="keyVltHPC01"

$certname = "HPCNodeCommCert"
$certSubName = "CN=fictal.com"


Write-Output "Variables: $tenantID $subscriptionID $location $resourceGroupName $keyVaultName $servPrincipalName $certname $certSubName"

Variables: 72f988bf-86f1-41af-91ab-2d7cd011db47 b67872ff-3d41-4494-a27c-a8ef9c3b0d00 WestUS2 rg-prdw2hpc001 kvt-prdw2hpc01 keyVltHPC01 HPCNodeCommCert CN=fictal.com


In [23]:
Connect-AzAccount -Tenant $tenantID -SubscriptionId $subscriptionID


Account               SubscriptionName      TenantId                             Environment
-------               ----------------      --------                             -----------
matperr@microsoft.com MWP_Prod_Subscription 72f988bf-86f1-41af-91ab-2d7cd011db47 AzureCloud



In [4]:
## Create Resource Group for the Key Vault
New-AzResourceGroup -Name $resourceGroupName -Location $location


ResourceGroupName : rg-prdw2hpc001
Location          : westus2
ProvisioningState : Succeeded
Tags              : 
ResourceId        : /subscriptions/b67872ff-3d41-4494-a27c-a8ef9c3b0d00/resourceGroups/rg-prdw2hpc0
                    01




In [21]:
## Create the KeyVault
# For testing, set Soft delete retention
New-AzKeyVault -VaultName $keyVaultName -ResourceGroupName $resourceGroupName -Location $location -Sku 'Standard' `
    -EnabledForDeployment -EnabledForTemplateDeployment   ` 
    -SoftDeleteRetentionInDays 7
    # Disabled -EnableRbacAuthorization due to the added step to app perms to add key
    # Disabled -EnabledForDiskEncryption not needed for HPC

## Need to enable Azure Services to use the vault
#Add-AzKeyVaultNetworkRule -VaultName $keyVaultName -VirtualNetworkResourceId $myNetworkResId -PassThru #-IpAddressRange "10.0.1.0/24"


Vault Name                          : kvt-prdw2hpc01
Resource Group Name                 : rg-prdw2hpc001
Location                            : WestUS2
Resource ID                         : /subscriptions/b67872ff-3d41-4494-a27c-a8ef9c3b0d00/resourceG
                                      roups/rg-prdw2hpc001/providers/Microsoft.KeyVault/vaults/kvt-
                                      prdw2hpc01
Vault URI                           : https://kvt-prdw2hpc01.vault.azure.net/
Tenant ID                           : 72f988bf-86f1-41af-91ab-2d7cd011db47
SKU                                 : Standard
Enabled For Deployment?             : True
Enabled For Template Deployment?    : True
Enabled For Disk Encryption?        : True
Enabled For RBAC Authorization?     : True
Soft Delete Enabled?                : True
Enabled Purge Protection?           : 
Soft Delete Retention Period (days) : 90
Enabled Purge Protection?           : 
Access Policies                     : 
                         

In [28]:
# Now create the Self-signed Certificate into keyvault
$Policy = New-AzKeyVaultCertificatePolicy -SecretContentType "application/x-pkcs12" -SubjectName "$certSubName" -IssuerName "Self" -ValidityInMonths 60 -ReuseKeyOnRenewal -KeyUsage DigitalSignature, KeyAgreement, KeyEncipherment -Ekus "1.3.6.1.5.5.7.3.1", "1.3.6.1.5.5.7.3.2"
Add-AzKeyVaultCertificate -VaultName $keyVaultName -Name "$certname" -CertificatePolicy $Policy

Add-AzKeyVaultCertificate: 
Line |
   3 |  Add-AzKeyVaultCertificate -VaultName $keyVaultName -Name "$certname"  …
     |  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | Operation returned an invalid status code 'Conflict'


In [27]:
# Get Certificate URL and Thumbprint
$keyVaultCert = Get-AzKeyVaultCertificate -Vaultname $keyVaultName -Name $certname

$certURL = $keyVaultCert.SecretId
$certThumb = $keyVaultCert.Thumbprint
Write-Host "Certificate information" -ForegroundColor Green
"Vault Name            : $keyVaultName"
"Vault Resource Group  : $resourceGroupName"
"Certificate URL       : $certURL"
"Certf Thumbprint      : $certThumb"

Certificate information
Vault Name            : kvt-prdw2hpc01
Vault Resource Group  : rg-prdw2hpc001
Certificate URL       : https://kvt-prdw2hpc01.vault.azure.net:443/secrets/HPCNodeCommCert/ef3988ffc0dd44dba16bdf3fd82c0dd1
Certf Thumbprint      : B163BCA1D8440325262F19B834231C7412A0C985


In [ ]:
######## WARNING #######
# Remove and then purge the keyvault
#####
Get-AzKeyVault -VaultName $keyVaultName
Remove-AzKeyVault -VaultName $keyVaultName 
Remove-AzKeyVault -VaultName $keyVaultName -InRemovedState -Location $location